---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
#import matplotlib.pyplot as plt 
def graph_identification():
    for G in P1_Graphs:
        avg_clu = nx.average_clustering(G)
        L = nx.average_shortest_path_length(G)

        #print('Avg. clustering:', avg_clu)
        #print('Avg. shortest path:', L)
        
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
        
        # Plots fraction of nodes as a function of degree (log - log scale)
        #plt.plot(degree_values, histogram, 'o')
        #plt.xlabel('Degree')
        #plt.ylabel('Fraction of nodes')
        #plt.xscale('log')
        #plt.yscale('log')
        #plt.show()
    
    
    # Preferential attachment model produces networks with a power law degree distribution (graph 1 and 4)
    # For small values of p, small world networks have small average shortest path
    # and high clustering coefficient (see graph 2 & 3 which have a high avg. clustering coefficient)
    answer = ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']
    return answer
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import mean_absolute_error
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import cross_val_score
    
    G.nodes(data=True)
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department')).astype(np.float32)
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary')).astype(np.float32)
    df['clustering'] = pd.Series(nx.clustering(G)).astype(np.float32)
    df['degree'] = pd.Series(G.degree()).astype(np.float32)
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G)).astype(np.float32)
    df['betweenness_centrality'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints = False)).astype(np.float32)
    df['closeness_centrality'] = pd.Series(nx.closeness_centrality(G, normalized = True)).astype(np.float32)
    
    train_df = df[df.ManagementSalary.notnull()].copy()
    test_df = df[df.ManagementSalary.isnull()].copy()

    y=train_df.ManagementSalary
    X_train = train_df.drop(['ManagementSalary'], axis=1)
    X_test = test_df.drop(['ManagementSalary'], axis=1)
    

    model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1,
             max_depth=5, random_state=0)
    
    model.fit(X_train, y)
    
    scores = cross_val_score(model, X_train, y,
                              cv=5,
                              scoring='roc_auc')
    #print('Roc_auc:', scores)
    #print("Roc_auc:", scores.mean())
    #print('Standard deviation:', scores.std())

    y_pred = model.predict_proba(X_test)
    ans = pd.Series(y_pred[:,1], index=X_test.index)
    
    return ans
salary_predictions()

1       0.000024
2       0.999735
5       0.999735
8       0.013448
14      0.002271
18      0.000609
27      0.001834
30      0.993349
31      0.136654
34      0.000358
37      0.000349
40      0.000363
45      0.000023
54      0.279136
55      0.946828
60      0.002218
62      0.999735
65      0.999754
77      0.000193
79      0.000193
97      0.000176
101     0.000024
103     0.918406
108     0.000325
113     0.001957
122     0.000017
141     0.003705
142     0.999735
144     0.000587
145     0.970359
          ...   
913     0.000243
914     0.000154
915     0.000014
918     0.000693
923     0.000195
926     0.002141
931     0.000106
934     0.000014
939     0.000013
944     0.000020
945     0.000195
947     0.000573
950     0.000940
951     0.000155
953     0.000030
959     0.000014
962     0.000013
963     0.000131
968     0.000433
969     0.000939
974     0.000223
984     0.000015
987     0.000461
989     0.000514
991     0.000591
992     0.000014
994     0.000021
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [25]:
def new_connections_predictions():
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.metrics import mean_absolute_error
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import cross_val_score
    
    G.edges(data=True)

    future_connections['ComN'] = future_connections.index.map(lambda emp: len(list(nx.common_neighbors(G, emp[0], emp[1]))))
    future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Resource_Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    
    new_df = future_connections.copy()

        
    train_df = new_df[new_df['Future Connection'].notnull()].copy()
    test_df = new_df[new_df['Future Connection'].isnull()].copy()
    
    
    y=train_df['Future Connection']
    X_train = train_df.drop(['Future Connection'], axis=1)
    X_test = test_df.drop(['Future Connection'], axis=1)
    
    model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1,
             max_depth=5, random_state=0)
    
    model.fit(X_train, y)
    
    scores = cross_val_score(model, X_train, y,
                              cv=5,
                              scoring='roc_auc')
    print('Roc_auc:', scores)
    print("Roc_auc:", scores.mean())
    #print('Standard deviation:', scores.std())



    y_pred = model.predict_proba(X_test)
    ans = pd.Series(y_pred[:,1], index=X_test.index)
        
    return ans
new_connections_predictions()

(107, 348)    0.026152
(542, 751)    0.012721
(20, 426)     0.567307
(50, 989)     0.012669
(942, 986)    0.012738
(324, 857)    0.013260
(13, 710)     0.172741
(19, 271)     0.148055
(319, 878)    0.012669
(659, 707)    0.012721
(49, 843)     0.012669
(208, 893)    0.012721
(377, 469)    0.010839
(405, 999)    0.018209
(129, 740)    0.015206
(292, 618)    0.045939
(239, 689)    0.012669
(359, 373)    0.015119
(53, 523)     0.030704
(276, 984)    0.012669
(202, 997)    0.012669
(604, 619)    0.094754
(270, 911)    0.012669
(261, 481)    0.061961
(200, 450)    0.832103
(213, 634)    0.012721
(644, 735)    0.130941
(346, 553)    0.012624
(521, 738)    0.012355
(422, 953)    0.017753
                ...   
(672, 848)    0.012669
(28, 127)     0.987017
(202, 661)    0.012624
(54, 195)     0.998990
(295, 864)    0.012721
(814, 936)    0.012721
(839, 874)    0.012738
(139, 843)    0.012721
(461, 544)    0.013960
(68, 487)     0.013363
(622, 932)    0.012721
(504, 936)    0.016848
(479, 528) 